In [18]:
# Librerías.
import pandas as pd
import Funciones as f
import unidecode
from docx import Document
from docx.shared import Inches
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

In [19]:
%%capture
%run "32. Análisis post hoc de cambios sumados.ipynb"

In [ ]:
def Graficar_Medias_Por_Grupo(Dataframe, Columna_Datos, Columna_Grupos, 
                              Titulo_Grafico, Escala_Y=None, 
                              Colores_Barras=None, Barras_Maximo=None,
                              Orden_Grupos=None, Nombre_df = "Generales",
                              Carpeta = "Graficos_Barras"):

    """
    Crea un gráfico de barras mostrando la media de una variable numérica 
    agrupada por categorías, incluyendo barras de error estándar.
    
    Parámetros:
    - Dataframe: DataFrame de pandas con los datos
    - Columna_Datos: str, nombre de la columna con valores numéricos
    - Columna_Grupos: str, nombre de la columna con categorías
    - Titulo_Grafico: str, título para el gráfico
    - Escala_Y: tuple opcional (min, max) para limitar el eje Y
    - Colores_Barras: list opcional con colores para cada barra
    - Barras_Maximo: int opcional, máximo número de barras a mostrar
    - Orden_Grupos: list opcional con el orden deseado de los grupos,
                    str 'alfabetico' para orden alfabético,
                    str 'por_media' para ordenar por valor de media (desc),
                    None para orden por tamaño de población (default)
    - Nombre_df: str, nombre del DataFrame para guardar el archivo.
    - Carpeta: str, nombre de la carpeta donde se guardará el gráfico.
    
    Ejemplo de uso:
    Graficar_Medias_Por_Grupo(df, 'Peso', 'Grupo_Edad', 
                              'Peso por Edad',
                              Orden_Grupos=['Joven', 'Adulto', 'Mayor'])
    
    """

    # Filtrar datos válidos (sin NaN en las columnas relevantes).
    Datos_Limpios = Dataframe.dropna(subset=[Columna_Datos, Columna_Grupos])
    
    # Calcular estadísticas por grupo incluyendo el conteo.
    Estadisticas_Grupo = Datos_Limpios.groupby(Columna_Grupos)[Columna_Datos].agg([
        'mean',   # Media
        'sem',    # Error estándar de la media
        'count'   # Número de observaciones
    ]).reset_index()
    
    # Renombrar columnas para mayor claridad.
    Estadisticas_Grupo.columns = [Columna_Grupos, 'Media', 'Error_Estandar', 'N']
    
    # Aplicar ordenamiento según el parámetro especificado.
    if Orden_Grupos is None:
        # Orden por tamaño de población (descendente) - comportamiento default.
        Estadisticas_Ordenadas = Estadisticas_Grupo.sort_values(
            'N', ascending=False
        ).copy()
        
    elif isinstance(Orden_Grupos, list):
        # Orden personalizado usando la lista proporcionada.
        # Crear un diccionario para mapear el orden.
        Orden_Mapeo = {grupo: i for i, grupo in enumerate(Orden_Grupos)}
        
        # Filtrar solo los grupos que están en la lista de orden.
        Estadisticas_Filtradas_Orden = Estadisticas_Grupo[
            Estadisticas_Grupo[Columna_Grupos].isin(Orden_Grupos)
        ].copy()
        
        # Aplicar el orden personalizado.
        Estadisticas_Filtradas_Orden['Orden_Indice'] = (
            Estadisticas_Filtradas_Orden[Columna_Grupos].map(Orden_Mapeo)
        )
        Estadisticas_Ordenadas = Estadisticas_Filtradas_Orden.sort_values(
            'Orden_Indice'
        ).drop('Orden_Indice', axis=1)
        
    elif Orden_Grupos == 'alfabetico':
        # Orden alfabético.
        Estadisticas_Ordenadas = Estadisticas_Grupo.sort_values(
            Columna_Grupos
        ).copy()
        
    elif Orden_Grupos == 'por_media':
        # Orden por valor de media (descendente).
        Estadisticas_Ordenadas = Estadisticas_Grupo.sort_values(
            'Media', ascending=False
        ).copy()
        
    else:
        print(f"Advertencia: Orden_Grupos '{Orden_Grupos}' no reconocido. "
              f"Usando orden por tamaño de población.")
        Estadisticas_Ordenadas = Estadisticas_Grupo.sort_values(
            'N', ascending=False
        ).copy()
    
    # Aplicar límite máximo de barras si se especifica.
    if Barras_Maximo is not None and Barras_Maximo > 0:
        Estadisticas_Finales = Estadisticas_Ordenadas.head(Barras_Maximo)
        Grupos_Eliminados = len(Estadisticas_Ordenadas) - Barras_Maximo
    else:
        Estadisticas_Finales = Estadisticas_Ordenadas
        Grupos_Eliminados = 0
    
    # Verificar si hay datos para graficar.
    if len(Estadisticas_Finales) == 0:
        print("Advertencia: No hay datos para graficar")
        return None
    
    # Configurar colores.
    if Colores_Barras is None:
        Colores_Usar = 'steelblue'
    else:
        # Si hay menos colores que grupos, repetir la lista.
        Num_Grupos = len(Estadisticas_Finales)
        if len(Colores_Barras) < Num_Grupos:
            Colores_Usar = (Colores_Barras * 
                           (Num_Grupos // len(Colores_Barras) + 1))[:Num_Grupos]
        else:
            Colores_Usar = Colores_Barras[:Num_Grupos]
    
    # Crear el gráfico.
    plt.figure(figsize=(10, 6))
    
    # Crear barras con barras de error.
    Barras = plt.bar(Estadisticas_Finales[Columna_Grupos], 
                     Estadisticas_Finales['Media'],
                     yerr=Estadisticas_Finales['Error_Estandar'],
                     capsize=5,
                     alpha=0.7,
                     color=Colores_Usar,
                     edgecolor='black',
                     linewidth=0.5)
    
    # Configurar etiquetas y título.
    plt.xlabel(Columna_Grupos, fontsize=12)
    plt.ylabel(f'Media de {Columna_Datos}', fontsize=12)
    plt.title(f'{Titulo_Grafico} de {Nombre_df}', fontsize=14, fontweight='bold')
    
    # Aplicar escala personalizada si se proporciona.
    if Escala_Y is not None:
        plt.ylim(Escala_Y)
    
    # Añadir valores encima de cada barra con el tamaño de muestra.
    for i, Barra in enumerate(Barras):
        Altura = Barra.get_height()
        N_Grupo = Estadisticas_Finales.iloc[i]['N']
        plt.text(Barra.get_x() + Barra.get_width()/2., Altura,
                f'{Altura:.2f}\n(n={N_Grupo})',
                ha='center', va='bottom', fontsize=9)
    
    # Mejorar el diseño.
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    
    # Mostrar información sobre grupos filtrados.
    if Grupos_Eliminados > 0:
        print(f"Se mostraron {Barras_Maximo} grupos según el criterio de ordenamiento.")
        print(f"Se omitieron {Grupos_Eliminados} grupo(s).")
    
    # Guardar en la carpeta creada.
    Ruta_Archivo = os.path.join(f"{Carpeta}/", 
                               f"{Carpeta}_{Columna_Datos}_{Nombre_df}.png")
    plt.savefig(Ruta_Archivo, dpi=300, bbox_inches='tight')

    # Mostrar el gráfico.
    #plt.show()
    
    # Retornar las estadísticas de los grupos mostrados.
    return Estadisticas_Finales                             

In [ ]:
Columnas_A_Graficar = [                   'CO_Item_5_Izq',
                                          'CO_Item_6_Izq',
                                          'CO_Item_9_Izq',
                                          'CO_Item_11_Izq',
                                          'CO_Item_16_Izq',
                                          'CO_Item_20_Izq',
                                          'CO_Item_24_Izq',
                                          'CO_Item_25_Izq',
                                          'CO_Item_27_Izq',
                                          'CO_Item_28_Izq',
                                          'CO_Item_5_Der',
                                          'CO_Item_6_Der',
                                          'CO_Item_9_Der',
                                          'CO_Item_11_Der',
                                          'CO_Item_16_Der',
                                          'CO_Item_20_Der',
                                          'CO_Item_24_Der',
                                          'CO_Item_25_Der',
                                          'CO_Item_27_Der',
                                          'CO_Item_28_Der',
                                          'CO_Item_3_Izq',
                                          'CO_Item_4_Izq',
                                          'CO_Item_7_Izq',
                                          'CO_Item_8_Izq',
                                          'CO_Item_10_Izq',
                                          'CO_Item_19_Izq', 
                                          'CO_Item_22_Izq',
                                          'CO_Item_23_Izq',
                                          'CO_Item_29_Izq',
                                          'CO_Item_30_Izq',
                                          'CO_Item_3_Der',
                                          'CO_Item_4_Der',
                                          'CO_Item_7_Der',
                                          'CO_Item_8_Der',
                                          'CO_Item_10_Der',
                                          'CO_Item_19_Der', 
                                          'CO_Item_22_Der',
                                          'CO_Item_23_Der',
                                          'CO_Item_29_Der',
                                          'CO_Item_30_Der',
                                          'Cambio_Op_Sum_Pro_Izq',
                                          'Cambio_Op_Sum_Pro_Der',
                                          'Cambio_Op_Sum_Con_Izq',
                                          'Cambio_Op_Sum_Con_Der',
                                          'Cambio_Op_Filt_Pro_Izq', 
                                          'Cambio_Op_Filt_Con_Izq', 
                                          'Cambio_Op_Filt_Con_Der']

In [22]:
%%capture
for Nombre_df, df in dfs_Finales.items():

    # Filtrar solo categorías políticas relevantes.
    df = df[df['Categoria_PASO_2023'].isin(['Left_Wing', 'Progressivism', 'Centre', 'Moderate_Right_A', 'Moderate_Right_B',
                                'Right_Wing_Libertarian'])].copy()
    
    # Crear carpeta si no existe.
    Carpeta_Graficos = "Graficos_Barras"
    os.makedirs(Carpeta_Graficos, exist_ok=True)

    # Graficar cada columna.    
    for Columna in Columnas_A_Graficar:
        if Columna in df.columns:  # Solo graficar si existe.
            Graficar_Medias_Por_Grupo(df, 
                                    Columna, 
                                    'Categoria_PASO_2023',
                                    Columna, 
                                    Escala_Y = None, 
                                    Colores_Barras = ['#f65058', '#0078bf', '#009cdd', 
                                                    '#f7d117', '#f7d117', '#753bbd'], 
                                    Barras_Maximo = 6, 
                                    Orden_Grupos = ['Left_Wing', 'Progressivism', 'Centre', 
                                                'Moderate_Right_A', 'Moderate_Right_B',
                                                'Right_Wing_Libertarian'],
                                    Nombre_df = Nombre_df)